# Testing energy minimization

In [ ]:
#import tyssue
import sys
#sys.path.append('/home/guillaume/Python/tyssue/src')
import pandas as pd
import numpy as np
import json
import matplotlib.pylab as plt
%matplotlib inline
from scipy import optimize

from tyssue.core.sheet import Sheet

from tyssue.utils.utils import set_data_columns

from tyssue.geometry.sheet_geometry import SheetGeometry as geom
from tyssue.dynamics.sheet_vertex_model import SheetModel as model
from tyssue.solvers.sheet_vertex_solver import Solver as solver

from tyssue.config.json_parser import load_default
import tyssue.dynamics.sheet_isotropic_model as iso
from tyssue.dynamics.sheet_isotropic_model import isotropic_relax



from tyssue.draw.mpl_draw import sheet_view
import tyssue.draw.mpl_draw as draw
from tyssue.io import hdf5
from tyssue.stores import load_datasets

In [ ]:
h5store = 'small_hexagonal.hf5'
datasets = hdf5.load_datasets(h5store,
                              data_names=['face', 'vert', 'edge'])
sheet = Sheet('emin', datasets)

sheet.set_geom('sheet')

geom.update_all(sheet)
sheet.vert_df.describe().head(3)


we define the adimentional contractility $\bar\Gamma = \Gamma/K_vA_0h_0^2$ and line tension
$\bar\Lambda = \Lambda /K_v (A_0^{3/2}h_0^2)$, where $h_0$ is such that $V_0 = A_0h_0$.


In [ ]:
sheet.face_df.describe().head(3)

In [ ]:

nondim_specs = load_default('dynamics', 'sheet')
dim_model_specs = model.dimentionalize(nondim_specs)

sheet.set_model('sheet', dim_model_specs)
sheet.grad_norm_factor = sheet.specs['settings']['grad_norm_factor']
sheet.nrj_norm_factor = sheet.specs['settings']['nrj_norm_factor']


isotropic_relax(sheet, nondim_specs)

In [ ]:
sheet.nrj_norm_factor

In [ ]:
Et, Ec, Ev = model.compute_energy(sheet, full_output=True)
energy = model.compute_energy(sheet, full_output=False)
print('Total energy: {}'.format(energy))

In [ ]:
sheet.face_df.head()

In [ ]:
from tyssue.draw.mpl_draw import plot_analytical_to_numeric_comp
fig, ax = plot_analytical_to_numeric_comp(sheet, model, geom, iso, nondim_specs)


In [ ]:
model.compute_energy(sheet) / sheet.face_df.is_alive.sum()

In [ ]:
isotropic_relax(sheet, nondim_specs)

fig, ax = sheet_view(sheet, ['z', 'x'])

In [ ]:
grad_t, grad_c, grad_v_srce, grad_v_trgt = model.compute_gradient(sheet, components=True)


In [ ]:
np.linalg.norm(grad_v_trgt.dropna(), axis=0).sum() / sheet.nrj_norm_factor

In [ ]:
grad_t.head()

In [ ]:
grad_c.head()

In [ ]:
hdf5.save_datasets('small_hexagonal.hf5', sheet)

In [ ]:
grad_i = model.compute_gradient(sheet, components=False)
grad_i.head()

In [ ]:
# geom.scale(sheet, 2, sheet.coords)
# geom.update_all(sheet)

In [ ]:
bck_lt = sheet.edge_df.line_tension.copy()
bck_ct = sheet.face_df.contractility.copy()
bck_ve = sheet.face_df.vol_elasticity.copy()


In [ ]:
# sheet.edge_df.line_tension = 0
# sheet.face_df.vol_elasticity = 0
# sheet.face_df.contractility = 0

In [ ]:
scale = 10
fig, ax = draw.plot_forces(sheet, geom, model, ['z', 'x'], scale)
fig.set_size_inches(10, 12)
for n, (vx, vy, vz) in sheet.vert_df[sheet.coords].iterrows():
    shift = 0.6 * np.sign(vy)
    ax.text(vz+shift-0.3, vx, str(n))

app_grad_specs = load_default('draw', 'sheet')['grad']
app_grad_specs.update({'color':'r'})
    
def draw_approx_force(ax=None):
    fig, ax = draw.plot_forces(sheet, geom, model,
                              ['z', 'x'], scaling=scale, ax=ax,
                              approx_grad=solver.approx_grad, **{'grad':app_grad_specs})
    fig.set_size_inches(10, 12)
    return fig, ax

## Uncomment bellow to recompute
fig, ax = draw_approx_force(ax=ax)
#fig

In [ ]:
sheet.edge_df.line_tension = bck_lt.copy()
sheet.face_df.contractility = bck_ct.copy()
sheet.face_df.vol_elasticity = bck_ve.copy()

http://scipy.github.io/devdocs/generated/scipy.optimize.check_grad.html#scipy.optimize.check_grad

In [ ]:

grad_err = solver.check_grad(sheet, geom, model)
grad_err /= sheet.vert_df.size


print("Error on the gradient (non-dim, per vertex): {:.3e}".format(grad_err))


In [ ]:
settings = {
    'minimize': {
        'options': {'disp':False,
                    'ftol':1e-4,
                    'gtol':1e-4},
        }
    }


res = solver.find_energy_min(sheet, geom, model, **settings)
print(res['success'])

In [ ]:
res['message']

In [ ]:
res['fun']/sheet.face_df.is_alive.sum()

In [ ]:
fig, ax = draw.plot_forces(sheet, geom, model, ['z', 'y'], 1)
fig.set_size_inches(10, 12)


In [ ]:
fig, ax = sheet_view(sheet, ['z', 'x'])